In [ ]:
# Clustering With K-Means

In [ ]:
# 本课和下课使用所谓的无监督学习算法。无监督算法不利用目标;相反，它们的目的是学习数据的某些属性，以某种方式表示特征的结构。
# 在用于预测的特征工程的上下文中，您可以将无监督算法视为一种“特征发现”技术。

# 聚类仅意味着根据数据点之间的相似程度将数据点分配给组。可以这么说，聚类算法使“一根羽毛的鸟群聚集在一起”。

# 例如，当用于特征工程时，我们可以尝试发现代表细分市场的客户组，或具有相似天气模式的地理区域。
# 添加聚类标签功能可以帮助机器学习模型理清空间或邻近度的复杂关系。

# 请务必记住，此群集功能是分类的。
# 在这里，它以标签编码（即整数序列）显示，就像典型的聚类算法一样;根据您的型号，单热编码可能更合适。

# 添加集群标签的动机是，集群会将跨功能的复杂关系分解为更简单的块。
# 然后，我们的模型可以一个接一个地学习更简单的块，而不必一次学习复杂的整体。这是一种“分而治之”的策略。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

df = pd.read_csv("../input/fe-course-data/housing.csv")
X = df.loc[:, ["MedInc", "Latitude", "Longitude"]]
X.head()

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X.head()

In [ ]:
sns.relplot(
    x="Longitude", y="Latitude", hue="Cluster", data=X, height=6,
)

In [ ]:
X["MedHouseVal"] = df["MedHouseVal"]
sns.catplot(x="MedHouseVal", y="Cluster", data=X, kind="boxen", height=6)

In [ ]:
# 一组特征的最佳分区取决于您使用的模型以及您尝试预测的内容，因此最好像任何超参数一样对其进行调整（例如，通过交叉验证）。

In [ ]:
# ex
# k 均值算法对尺度很敏感。这意味着我们需要深思熟虑如何以及是否重新扩展我们的功能，因为我们可能会根据我们的选择得到非常不同的结果。
# 根据经验，如果特征已经可以直接比较（例如不同时间的测试结果），那么您就不想重新缩放。
# 另一方面，不在可比比例上的特征（如身高和体重）通常会从重新缩放中受益。
# 有时，选择并不明确。在这种情况下，您应该尝试使用常识，记住具有较大值的特征将受到更重的权重。


In [ ]:
# Scaling Features
# Consider the following sets of features. For each, decide whether:

# they definitely should be rescaled,
# they definitely should not be rescaled, or
# either might be reasonable

""" 不可以，因为重新缩放会扭曲“纬度”和“经度”描述的自然距离。
任何一种选择都是合理的，但由于房屋的居住面积往往每平方英尺更有价值，因此重新缩放这些特征是有意义的，
这样地块面积在聚类中的权重就不会与其对 SalePrice 的影响不成比例，如果这是您试图预测的。
是的，因为这些没有可比的单位。如果不重新缩放，汽车中的车门数量（通常为 2 或 4 个）与其马力（通常为数百个）相比，重量可以忽略不计。 """

# 您应该从中吸取的教训是，是否以及如何重新缩放特征的决定很少是自动的 - 它通常取决于有关您的数据和您尝试预测的内容的一些领域知识。
# 通过交叉验证比较不同的重新缩放方案也会有所帮助。（您可能想查看 scikit-learn 中的预处理模块，了解它提供的一些重新缩放方法。

In [ ]:
X = df.copy()
y = X.pop("SalePrice")


# YOUR CODE HERE: Define a list of the features to be used for the clustering
features = ["LotArea","TotalBsmtSF","FirstFlrSF","SecondFlrSF","GrLivArea"]


# Standardize
X_scaled = X.loc[:, features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)


# YOUR CODE HERE: Fit the KMeans model to X_scaled and create the cluster labels
kmeans = KMeans(n_clusters=10, random_state=0)
X["Cluster"] = kmeans.fit_predict(X_scaled)

In [ ]:
Xy = X.copy()
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["SalePrice"] = y
sns.relplot(
    x="value", y="SalePrice", hue="Cluster", col="variable",
    height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=3,
    data=Xy.melt(
        value_vars=features, id_vars=["SalePrice", "Cluster"],
    ),
);

In [ ]:
# 记住每做完一次聚类就评估一下分数
score_dataset(X, y)

In [ ]:
# Cluster-Distance Features
# Now add the cluster-distance features to your dataset. 
# You can get these distance features by using the fit_transform method of kmeans instead of fit_predict.
kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)


# YOUR CODE HERE: Create the cluster-distance features using `fit_transform`
X_cd = kmeans.fit_transform(X_scaled)


# Label features and join to dataset
X_cd = pd.DataFrame(X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
X = X.join(X_cd)